# The Battle of Neighborhoods Week 2

## Import Libaries

In [15]:
import pandas as pd
import folium
import geopy.geocoders as Nominatim
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from sklearn.cluster import KMeans 
import matplotlib.cm as cm

### There is no data what I wanted on the internet so,I will create excel file that contains the location where I interested

In [2]:
path = r'C:\Users\saharath\Desktop\Python for Data science\capstone proj\north_bkk.xlsx'

In [3]:
df_read = pd.read_excel(path,index_col=False)
df_read.head(10)

,District,Sub,Population,Latitude,Longitude,Salary
0,Chatuchak,Lat Yao,41814,13.843484,100.561663,16475
1,Chatuchak,Sena Nikhom,20417,13.840416,100.585228,13975
2,Chatuchak,Chan Kasem,38701,13.823315,100.582452,16557
3,Chatuchak,Chom Phon,32288,13.810020,100.566143,18180
4,Chatuchak,Chatuchak,23385,13.816970,100.560225,14284
5,Lat Phrao,Lat Phrao,91456,13.812210,100.609746,16966
6,Lat Phrao,Chorakhe Bua,27118,13.836543,100.604940,17185
7,Bang Sue,Bang Sue,82570,13.806294,100.531051,14292
8,Bang Sue,Wong Sawang,43740,13.829567,100.527158,15609


In [4]:
df_bkk = df_read

In [5]:
#Reanem column 'Sub' to 'SubDistrict'
df_bkk.rename(columns={'Sub' : 'SubDistrict'},inplace=True)

## Get Northern Bangkok Latitude&Longitude via Google maps

To visualize the Northern Bangkok, I picked Lat&Long via google maps

In [6]:
BKK_Latitude = 13.833037
BKK_Longitude = 100.571368

## Create a Folium Map

Visualize SubDistrict by Folium

In [7]:
bkk_map = folium.Map(location=[BKK_Latitude,BKK_Longitude],zoom_start=13)

#Create center circle of district
for a,b in df_bkk.iterrows():
    folium.Circle(
            radius=200,
            location=[b['Latitude'],b['Longitude']],
            popup=b['SubDistrict'],
            color='Blue',
            fill=False,
            ).add_to(bkk_map)
 

In [8]:
bkk_map

## Get venues via Foursquare API

In [9]:
CLIENT_ID = 'PDDTMSTCLS1QATQOFCVRR3VUDAAQXKWJCYO1VVY2UNRL5TJF' # your Foursquare ID
CLIENT_SECRET = 'F5KZR01TRG413J5BOWFPJJZCLCAHM4HJWL4FCWNQV5OW2FGG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PDDTMSTCLS1QATQOFCVRR3VUDAAQXKWJCYO1VVY2UNRL5TJF
CLIENT_SECRET:F5KZR01TRG413J5BOWFPJJZCLCAHM4HJWL4FCWNQV5OW2FGG


In [10]:

LIMIT = 100
radius = 1000

## Get nearby 100 venues

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['SubDistrict', 
                  'SubDistrict Latitude', 
                  'SubDistrict Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
df_bkk.head()

,District,SubDistrict,Population,Latitude,Longitude,Salary
0,Chatuchak,Lat Yao,41814,13.843484,100.561663,16475
1,Chatuchak,Sena Nikhom,20417,13.840416,100.585228,13975
2,Chatuchak,Chan Kasem,38701,13.823315,100.582452,16557
3,Chatuchak,Chom Phon,32288,13.810020,100.566143,18180
4,Chatuchak,Chatuchak,23385,13.816970,100.560225,14284


In [13]:
bkk_venues = getNearbyVenues(names=df_bkk['SubDistrict'],
                                   latitudes=df_bkk['Latitude'],
                                   longitudes=df_bkk['Longitude'])

Lat Yao


ProxyError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=PDDTMSTCLS1QATQOFCVRR3VUDAAQXKWJCYO1VVY2UNRL5TJF&client_secret=F5KZR01TRG413J5BOWFPJJZCLCAHM4HJWL4FCWNQV5OW2FGG&v=20180605&ll=13.843484,100.561663&radius=1000&limit=100 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 Proxy Authentication Required')))

In [ ]:
#Check size
print(bkk_venues.shape)
#bkk_venues[bkk_venues['Venue Category']=='Coffee Shop']
bkk_venues.tail()

In [ ]:
#Count venues and group by District
bkk_venues.groupby('SubDistrict').count()

In [ ]:
print('There are {} uniques categories.'.format(len(bkk_venues['Venue Category'].unique())))

## Analyze each district

1.Visualize population and average salary

In [ ]:
fig , ax = plt.subplots(1, 2, sharex = True,sharey = False)
pop_bar = df_bkk.set_index('SubDistrict')
sal_bar = df_bkk.set_index('SubDistrict')
g1=pop_bar['Population'].plot(kind='bar',title='Population',color='Blue',figsize = (12,5),alpha=0.5,fontsize=12,ax=ax[0])
g2=sal_bar['Salary'].plot(kind='bar',title='Average Salary',color='Green',alpha=0.5,fontsize=12, ax=ax[1])

'Lat Phrao' has the the most population with number over 80,000. The average salary 'Chom Phon' is 1st and followed by 'Chorake Bua' and 'Lat Phrao'

In [ ]:
bkk_venues.head()
# = bkk_venues[bkk_venues['Venue Category']=='Coffee Shop']

In [ ]:
nearby_map = folium.Map(location=[BKK_Latitude,BKK_Longitude],zoom_start=13)

    
#Create center circle of district
for a,b in bkk_venues.iterrows():
    folium.Circle(
            radius=100,
            location=[b['Venue Latitude'],b['Venue Longitude']],
            popup=b['Venue'],
            color='Grey',
            alpha=0.5,
            fill=False,
            ).add_to(nearby_map)    

nearby_map

In [ ]:
# one hot encoding
bkk_onehot = pd.get_dummies(bkk_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bkk_onehot['SubDistrict'] = bkk_venues['SubDistrict'] 

# move neighborhood column to the first column
fixed_columns = [bkk_onehot.columns[-1]] + list(bkk_onehot.columns[:-1])
bkk_onehot = bkk_onehot[fixed_columns]

bkk_onehot.tail()

In [ ]:
bkk_grouped = bkk_onehot.groupby('SubDistrict').mean().reset_index()
bkk_grouped

In [ ]:
num_top_venues = 5

for hood in bkk_grouped['SubDistrict']:
    print("----"+hood+"----")
    temp = bkk_grouped[bkk_grouped['SubDistrict'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['SubDistrict']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
SubDistrict_venues_sorted = pd.DataFrame(columns=columns)
SubDistrict_venues_sorted['SubDistrict'] = bkk_grouped['SubDistrict']

for ind in np.arange(bkk_grouped.shape[0]):
    SubDistrict_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bkk_grouped.iloc[ind, :], num_top_venues)

SubDistrict_venues_sorted.head()

# Cluster SubDistrict

In [14]:
# set number of clusters
kclusters = 4

bkk_grouped_clustering = bkk_grouped.drop('SubDistrict', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bkk_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

NameError: name 'bkk_grouped' is not defined

In [ ]:
bkk_data = df_bkk.drop(['Population','Salary'],axis=1)

In [ ]:
bkk_data

In [ ]:
# add clustering labels
SubDistrict_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bkk_merged = bkk_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bkk_merged = bkk_merged.join(SubDistrict_venues_sorted.set_index('SubDistrict'), on='SubDistrict')

bkk_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[BKK_Latitude, BKK_Longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bkk_merged['Latitude'], bkk_merged['Longitude'], bkk_merged['SubDistrict'], bkk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

In [ ]:
bkk_merged.loc[bkk_merged['Cluster Labels'] == 0, bkk_merged.columns[[1] + list(range(5, bkk_merged.shape[1]))]]

In [ ]:
bkk_merged.loc[bkk_merged['Cluster Labels'] == 1, bkk_merged.columns[[1] + list(range(5, bkk_merged.shape[1]))]]

In [ ]:
bkk_merged.loc[bkk_merged['Cluster Labels'] == 2, bkk_merged.columns[[1] + list(range(5, bkk_merged.shape[1]))]]

In [ ]:
bkk_merged.loc[bkk_merged['Cluster Labels'] == 3, bkk_merged.columns[[1] + list(range(5, bkk_merged.shape[1]))]]